In [1]:
# In the following you can define your desired output. Current options:
# per_residue embeddings
# per_protein embeddings
# secondary structure predictions

# Replace this file with your own (multi-)FASTA
# Headers are expected to start with ">";
seq_path = "./protT5/example_seqs.fasta"

# whether to retrieve embeddings for each residue in a protein 
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = True 
per_residue_path = "./protT5/output/per_residue_embeddings.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings 
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = False
per_protein_path = "./protT5/output/per_protein_embeddings.h5" # where to store the embeddings

# whether to retrieve secondary structure predictions
# This can be replaced by your method after being trained on ProtT5 embeddings
sec_struct = False
sec_struct_path = "./protT5/output/ss3_preds.fasta" # file for storing predictions

# make sure that either per-residue or per-protein embeddings are stored
assert per_protein is True or per_residue is True or sec_struct is True, print(
    "Minimally, you need to active per_residue, per_protein or sec_struct. (or any combination)")


In [2]:
!nvidia-smi

Sun Jul  9 15:19:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   54C    P8    34W / 290W |    381MiB /  8192MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#@title Import dependencies and check whether GPU is available. { display-mode: "form" }
from transformers import T5EncoderModel, T5Tokenizer
import torch
import h5py
import time
import gc
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using {}".format(device))

/home/nayeem/anaconda3/envs/dta/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda:0


In [4]:
#@title Load ProtT5 in half-precision. { display-mode: "form" }
# Load ProtT5 in half-precision (more specifically: the encoder-part of ProtT5-XL-U50) 
def get_T5_model():
    model = T5EncoderModel.from_pretrained("../protT5/protT5_checkpoint/", torch_dtype=torch.float16)
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False ) 

    return model, tokenizer

In [5]:
#@title Generate embeddings. { display-mode: "form" }
# Generate embeddings via batch-processing
# per_residue indicates that embeddings for each residue in a protein should be returned.
# per_protein indicates that embeddings for a whole protein should be returned (average-pooling)
# max_residues gives the upper limit of residues within one batch
# max_seq_len gives the upper sequences length for applying batch-processing
# max_batch gives the upper number of sequences per batch
def get_embeddings( model, tokenizer, seqs, per_residue, per_protein, sec_struct, 
                   max_residues=15000, max_seq_len=1200, max_batch=100 ):

    # if sec_struct:
    #   sec_struct_model = load_sec_struct_model()

    results = {"residue_embs" : dict(), 
               "protein_embs" : dict(),
               "sec_structs" : dict() 
               }

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed 
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len 
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
#             print(len(seqs))
#             print(seq_lens)
            batch = list()

            # print(n_res_batch)
            # print(len(seqs))

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs,
                                                        add_special_tokens = True,
                                                        max_length = max_seq_len, 
                                                        padding = 'max_length',
                                                        truncation = True,
                                                        return_tensors = 'pt')
            input_ids      = token_encoding['input_ids'].to(device)
            # print(f'Shape of input ids is {input_ids.shape}')
            attention_mask = token_encoding['attention_mask'].to(device)
            # print(f'Shape of input ids is {input_ids.shape}')
            
            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError as e:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            # if sec_struct: # in case you want to predict secondary structure from embeddings
            #   d3_Yhat, d8_Yhat, diso_Yhat = sec_struct_model(embedding_repr.last_hidden_state)


            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                # slice off padding --> batch-size x seq_len x embedding_dim  
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if sec_struct: # get classification results
                    results["sec_structs"][identifier] = torch.max( d3_Yhat[batch_idx,:s_len], dim=1 )[1].detach().cpu().numpy().squeeze()
                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(results["residue_embs"]) if per_residue else passed_time/len(results["protein_embs"])
    # print('\n############# EMBEDDING STATS #############')
    # print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    # print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    # print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
    #     passed_time/60, avg_time ))
    # print('\n############# END #############')

    return results

In [8]:
last_processed = -1
with open('checkpoint.txt') as f:
  last_processed = int(f.read())

In [9]:
last_processed

-1

In [8]:
def update_checkpoint(val: int):
  with open('checkpoint.txt', 'w') as f:
    f.write(str(val))

In [63]:
from collections import OrderedDict
import pandas as pd

df_train = pd.read_csv('./data/kiba/raw/data_train.csv', usecols=['target_sequence'])
df_test = pd.read_csv('./data/kiba/raw/data_test.csv', usecols=['target_sequence'])
df = pd.concat([df_train, df_test], axis=0)
df.drop_duplicates(inplace=True)
seqs = df['target_sequence'].to_dict(OrderedDict)

In [6]:
# Load the encoder part of ProtT5-XL-U50 in half-precision (recommended)
model, tokenizer = get_T5_model()

In [60]:
keys = [key for key in seqs.keys()]
seq_num_to_process = 1
seq_to_write = 1000
index = (last_processed + 1)* seq_to_write
key_list = keys[index: ]
index

0

In [53]:
key_list[0]

0

In [26]:
last_processed

-1

In [64]:
len(key_list)

229

In [55]:
import numpy as np
from tqdm import trange, tqdm

In [65]:
buffer_dict = {}
write_counter = last_processed + 1
max_length = 1200

for i in tqdm(range(0, len(key_list), seq_num_to_process)):
    # print(f"epoch{i}/{len(key_list)//seq_num_to_process}")
    seq_keys = key_list[i : i+seq_num_to_process]

    temp_data_dic = {}

    for seq_key in seq_keys:
        temp_data_dic[seq_key] = seqs[seq_key]


    results = get_embeddings( model, tokenizer, temp_data_dic,
                      per_residue, per_protein, sec_struct, max_seq_len=max_length, max_batch=10)

    embedding = results['residue_embs'][seq_keys[-1]]
    sequence = seqs[seq_keys[-1]]
    buffer_dict.update({sequence:torch.from_numpy(embedding)})

    del results
    gc.collect()

100%|██████████| 229/229 [01:17<00:00,  2.94it/s]


In [66]:
torch.save(buffer_dict, './data/kiba/raw/prot5.pth')

In [ ]:
results['protein_embs']

In [ ]:
# Load example fasta.
seqs = read_fasta( seq_path )

In [ ]:
seqs['sp|P83302|3SX1_OPHHA Neurotoxin Oh9-1 OS=Ophiophagus hannah OX=8665 PE=1 SV=1'][0]

In [ ]:
seq_emb_2 = results['protein_embs']['sp|P83302|3SX1_OPHHA Neurotoxin Oh9-1 OS=Ophiophagus hannah OX=8665 PE=1 SV=1']

In [ ]:
print(type(seq_emb_2))

In [ ]:
import numpy as np
import h5py
import pandas as pd

hf = h5py.File('/content/protT5/output/per_protein_embeddings.h5', 'r')
n1 = np.array(hf["sp|P04637|P53_HUMAN Cellular tumor antigen p53 OS=Homo sapiens OX=9606 GN=TP53 PE=1 SV=4"][:]) #dataset_name is same as hdf5 object name 

print(type(n1))

In [ ]:
print(len(n1))

In [ ]:
unpadded_df = pd.read_csv('/content/drive/MyDrive/Updated_dataset/SARSCOV2_SpikeProtSeqDB_Mod.csv')

In [ ]:
len(unpadded_df)

In [ ]:
unpadded_df['Accession'].value_counts

In [ ]:
unpadded_df_ = pd.read_csv('/content/drive/MyDrive/Updated_dataset/Modified.csv', nrows=100)

In [ ]:
unpadded_df_.iloc[0]

In [ ]:
type(data_dict['EPI_ISL_1180899'])

In [ ]:
import pandas as pd
import sys
import h5py
import numpy as np

In [ ]:
key = []
cols = [i for i in range(1024)]
df = pd.DataFrame(columns=cols)
csv_path = f'/content/drive/MyDrive/Updated_dataset/Code + data updated/source_code/data/independent_set/mutated.csv'
print(csv_path)

fpath = f'/content/drive/MyDrive/Updated_dataset/Code + data updated/source_code/data/independent_set/mutated_0.h5'

hf = h5py.File(fpath, 'r')

keys = list(hf.keys())

for k in keys:
  arr = np.array(hf[k])
  df2 = pd.DataFrame(arr.reshape(1, -1), columns=cols)
  df = pd.concat([df, df2])
  key.append(k)

k_s = pd.Series(key)
df.insert(loc=0, column='Accession ID', value=k_s)
print(df.shape)
df.to_csv(csv_path)

/content/drive/MyDrive/Updated_dataset/Code + data updated/source_code/data/independent_set/mutated.csv
(1000, 1025)
